<a href="https://colab.research.google.com/github/sekadiv/AI-ML/blob/main/Apriori%2BAlgorithm%2B_%2BGraded%2BQuestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PART A: Frequent Itemset Generation

In [1]:
# Importing the libraries and the dataset

import numpy as np 
import pandas as pd

Market_Data = pd.read_csv('Market_Basket_Optimisation_demo.csv',index_col=None, header = None ) # Use your local path here

Market_Data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Converting the Market Dataset into a nested list
dataSet = []

for index, transaction in Market_Data.iterrows():
    cleaned_transaction = transaction[~transaction.isnull()].tolist()
    dataSet.append(cleaned_transaction)

#### 1. The createItem function

In [9]:
# For the given dataset writing a function to return the list of distinct items in the dataset

def createItem(dataSet):
    
    itemList = []
    
    for transaction in dataSet:
        for item in transaction:
            if not [item] in itemList:
                
                 # creating unique single lists in Itemlist. ie list of all items
                itemList.append([item])
                
    itemList.sort()
    
    return list(map(frozenset, itemList))

#### 2. The scanData function

In [10]:
# Returns an Itemset and dictionary with Support values

def scanData(data, itemsetk, minSupport):
    
    tempDict = {}
    
    for transaction in data:
        for item in itemsetk:
            if item.issubset(transaction):
                if not item in tempDict: 
                    tempDict[item]=1 # tempDict contains number of all items
                else: 
                    tempDict[item] += 1
    
    numItems = float(len(data))
    freqItemset = []
    supportDict = {}
    
    for key in tempDict:
        support = tempDict[key]/numItems 
        
        if support >= minSupport:
            freqItemset.insert(0,key) # freqItemset contains all frequent items
        supportDict[key] = support # contains support of all items
    
    return freqItemset, supportDict

#### 3. The itemSetGenerator function

In [12]:
# Creating Higher order Itemsets

def itemSetGenerator(itemsetk, k):
 
    higherOrderitemset = []
    lenitemsetk = len(itemsetk)
    
    
    for i in range(lenitemsetk):
        for j in range(i+1, lenitemsetk): 
            L1 = sorted(list(itemsetk[i]))[:k-2] 
            L2 = sorted(list(itemsetk[j]))[:k-2] 
            
            # Two frequent itemsets of order k are merged only if their k-1 itemsets are identical
            if L1 == L2:
                higherOrderitemset.append(itemsetk[i] | itemsetk[j]) # Performing set union creates itemset with n+1 items
               
    return higherOrderitemset

#### 4. Frequent Itemsets Generation

In [13]:
def apriori(dataSet, minSupport):
    
    itemList = createItem(dataSet) # Creating frozenset of items
    
    
    # Generating all the frequent 1-itemsets and the support of those items
    freqItemset1, supportDict = scanData(dataSet, itemList, minSupport)
    freqItemsets = [freqItemset1]
    k = 2 
    
    while (len(freqItemsets[k-2]) > 0): # Incrementing k until we no longer find any kth order itemsets
        
        itemsetk = itemSetGenerator(freqItemsets[k-2], k) # Generating itemsets of order k
        
        # Generating the frequent itemset for the kth order and support for each of these itemsets
        freqItemsetk, supportDictk = scanData(dataSet, itemsetk, minSupport) 
        
        supportDict.update(supportDictk)
        freqItemsets.append(freqItemsetk)
        
        k += 1
    return freqItemsets, supportDict

Calculate the maximum possible two itemsets.
- 140
- 120
- 7140
- 5240

In [16]:
# Identify the distinct items involved in the dataset

unique_list=createItem(dataSet)
len(unique_list)

120

In [22]:
# Calculate the number of possible two itemsets

ff=itemSetGenerator(unique_list,2)

In [23]:
len(ff)

7140

In [ ]:
ff

 Identify the support of itemset {'eggs','mineral water','spaghetti'} [ Round the value upto three decimal points]

- 0.022
- 0.034
- 0.014
- 0.052

Identify the number of transaction where items eggs, mineral water and spaghetti are bought together

- 165
- 255
- 392
- 107

In [35]:
# Identify the support of itemset {'eggs','mineral water','spaghetti'} 
jj,kk=apriori(dataSet,0.02)


In [94]:
kk[frozenset({'eggs', 'mineral water', 'spaghetti'})]

0.014264764698040262

In [102]:
#Identify the number of transaction where items eggs, mineral water and spaghetti are bought together
len(dataSet) * kk[frozenset({'eggs', 'mineral water', 'spaghetti'})]

107.0

Identify the number of frequent itemsets involving both chocolate and mineral water if minSupport is defined as 0.001

- 244
- 502
- 343
- 156

In [112]:
# Generate the frequent itemsets when minSupport = 0.001


In [117]:
#First, you generate all the frequent itemsets, and then you write a function to check if a particular frequent itemset contains the items chocolate and mineral water.

# Generate the frequent itemsets when minSupport = 0.001

freqItemsets, supportDict= apriori(dataSet,minSupport=0.001)
 

# Code to find the number of frequent itemsets containing both chocolate and mineral water


x = frozenset({'chocolate', 'mineral water'})
count = 0

for itemsetsK in freqItemsets:
    for items in itemsetsK:
        if items.intersection(x) == frozenset({'chocolate', 'mineral water'}) :
            count = count +1
                   
print(count)

245


In [116]:
kk1[frozenset({'chocolate', 'mineral water'})]

0.05265964538061592

In [115]:
len(dataSet) * kk1[frozenset({'chocolate', 'mineral water'})]

395.0

In [ ]:
# Write a code to find the number of frequent itemset containing both chocolate and mineral water




### PART B: Rule Generation

#### 5. The ‘calcConf’ function

In [ ]:
def calcConf(freqSet, H, supportDict, bigRuleList, minConf):

    prunedH = []
    
    for conseq in H:
        
        conf = supportDict[freqSet]/supportDict[freqSet - conseq] # calculate confidence
        
        if conf >= minConf:
            bigRuleList.append((freqSet-conseq, conseq, conf))
            print(freqSet-conseq, '--->', conseq, 'confidence = ', conf)
            prunedH.append(conseq)
#             
    return prunedH

#### 6. The rulesFromConseq function


In [ ]:
def rulesFromConseq(freqSet, H, supportDict, bigRuleList, minConf):

    m = len(H[0]) # Order of the consequent while generating the rules
         
    H = calcConf(freqSet, H, supportDict, bigRuleList, minConf)
    if len(H)>1: # For len(H)<=1, you cannot generate higher order cadnidates
        
        # creating higher order candidates
        Hmp1 = itemSetGenerator(H, m+1) 
        
        if Hmp1 == []: # This will happen if higher order consequent itemsets are not possible
            # Hmp1 will be an empty list if the itemsets in H don't satisfy the condition for merging
            return 0
        
        if (len(Hmp1[0]) < len(freqSet)):
            # Generate rules while the order of the itemsets in Hmp1 is less than the number of items in the frequent itemset
            rulesFromConseq(freqSet, Hmp1, supportDict, bigRuleList, minConf)

#### 7. The generateRules function


In [ ]:
def generateRules(freqItemsets, supportDict, minConf):  #supportDict is a dictionary coming from scanData
    bigRuleList = []
    for i in range(1, len(freqItemsets)): # Only get the sets with two or more items
        for freqSet in freqItemsets[i]:
            H1 = [frozenset([item]) for item in freqSet]  
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportDict, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportDict, bigRuleList, minConf)
    return bigRuleList 

In order to  identify interesting rules from the dataset, you use constraints on metrics such as support, confidence, lift. Implement the measure lift and find out the rules with minSupport=0.05, minConf = 0.2 and lift > 1.2

Ref Link - https://en.wikipedia.org/wiki/Lift_(data_mining)

- 4
- 6
- 18
- 0

In [ ]:
# In the code for generating rules, implement the computation of Lift



In [ ]:
# Generate the rules when minSupport = 0.05, minConfidence = 0.2 and lift >1.2 



In [ ]:
# count the number of rules



Imagine that for some threshold support value, the itemset  {'eggs','chocolate','spaghetti'} is found to be a frequent itemset. Now for the rule {'chocolate'} --> {'spaghetti', 'eggs'} to be valid what can be the max value of minConfidence? [More than one answer type question]

- 0.08
- 0.04
- 0.06
- 0.01

In [ ]:
#Itendify the support of {'eggs', 'mineral water', 'chocolate'}




In [ ]:
# What will be the confidence of the rule {'chocolate'} --> {'spaghetti', 'eggs'}


